In [ ]:
!pip install gspread --upgrade
!pip install gspread_dataframe --upgrade
!pip install kora

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Localization'

import pandas as pd
import os
from google.colab import auth
auth.authenticate_user()
from google.auth import default
creds, _ = default()
import gspread
gc = gspread.authorize(creds)
import gspread_dataframe as gd

from kora.xattr import get_id

# Set paths to input and output folders
input_folder = 'FromUnity'
output_folder = 'ToUnity'

# List of tuples containing language name and Google Translate language code
languages = [('Chinese (Simplified)(zh-Hans)', 'zh-Hans'),
             ('French(fr)', 'fr'),
             ('German(de)', 'de'),
             ('Hindi(hi)', 'hi'),
             ('Italian(it)', 'it'),
             ('Japanese(ja)', 'ja'),
             ('Korean(ko)', 'ko'),
             ('Portuguese(pt)', 'pt'),
             ('Russian(ru)', 'ru'),
             ('Spanish(es)', 'es'),
             ('Thai(th)', 'th'),
             ('Ukrainian(uk)', 'uk')]

# Loop through each file in input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'): # check if file is a CSV
        # Read CSV file into pandas dataframe
        df = pd.read_csv(os.path.join(input_folder, filename))

        # Check if CSV corresponds to a string table
        if "Key" in df.columns[0]:
          # Fill rows
          for index, row in df.iterrows():
            for language, code in languages:
              df.at[index, language] = '=iferror(GOOGLETRANSLATE(F{0}, "en", "{1}"))'.format(index+2, code)

        # Otherwise, check if CSV corresponds to a dialogue database
        elif 'Conversation ID' in df.columns[0]:
          translated_text_header = df.columns[3]
          language_code = translated_text_header[translated_text_header.index('[') + 1 : translated_text_header.index(']')]

          # Fill rows
          for index, row in df.iterrows():
            df.iat[index, 3] = '=iferror(GOOGLETRANSLATE(C{0}, "en", "{1}"))'.format(index+2, language_code)

        # Otherwise, CSV corresponds to miscellaneous text
        elif 'Field' in df.columns[0]:
          print("TODO")

        else:
          print(f"FAILURE: {filename} format invalid")
          continue

        # Write modified dataframe to output CSV file
        output_filename = os.path.join(output_folder, filename)
        df.to_csv(output_filename, index=False, header=True)

        sh = gc.create(filename, get_id(output_folder))
        worksheet = sh.get_worksheet(0)
        gd.set_with_dataframe(worksheet, df)

        !rm $output_filename

        # Print receipt of success!
        print(f"SUCCESS: {filename}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1WSKRK8J2AZ3c-UVGPhGXQvj5pzO-tKkK/Localization
SUCCESS: Act1_Main_DialogueDB_it.csv
SUCCESS: Act1_Main_DialogueDB_ko.csv


FileNotFoundError: ignored